<a href="https://colab.research.google.com/github/alexlinapp/python_tools_practice/blob/main/number_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [2]:
# Define transforms to convert PIL images to tensors
transform = transforms.ToTensor()

# Download and load MNIST training data
mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Download and load MNIST test data
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

100%|██████████| 9.91M/9.91M [00:00<00:00, 16.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 505kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.04MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.00MB/s]


In [72]:
dl_train = DataLoader(mnist_train, batch_size=16, shuffle=True)
dl_valid = DataLoader(mnist_test, batch_size=16, shuffle=True)
xb, yb = next(iter(dl_train))
xb.shape, yb.shape
xb = xb.view(xb.size(0), -1)
#yb = yb.view(yb.size(0), -1)
xb.shape,yb.shape
#xb[0], yb[0], yb.shape
# preds = linear1(xb[:4])
# loss = mnist_loss(preds, yb[:4])
# loss



(torch.Size([16, 784]), torch.Size([16]))

In [63]:
class BasicOptimizer:
  def __init__(self, params, learnrate):
    self.params = list(params)
    self.learnrate = learnrate

  def step(self, *args, **kwargs):
    for p in self.params:
      #p.data -= p.grad.data * self.learnrate #using .data so it isn't calculated into gradient
      if p.grad is not None:
            with torch.no_grad():
                p.sub_(self.learnrate * p.grad)
  def zero_grad(self, *args, **kwargs):
    for p in self.params:
      if p.grad is not None:
        p.grad.zero_()

In [65]:
def init_params(size, std=1.0):
  return (torch.randn(size)*std).requires_grad_()

# own softmax function for practice, use torch.softmax(logits, dim=1)
# subtracting x.max(dim=1).values for numerical stability. Dividing by e^{-m} / e^{-m}
def softmax(x):
  e_x = torch.exp(x - x.max(dim=1, keepdim=True).values)
  return e_x / e_x.sum(dim = 1, keepdim=True)
def mnist_loss(prediction, target):
  eps = 1e-7
  prediction = softmax(prediction)
  prediction = prediction.clamp(min=eps)
  idx = range(target.size(0))
  prediction = prediction[idx, target]
  # print("This is prediction", prediction)
  # print("THis is log torch: ", -torch.log(prediction))
  return (-torch.log(prediction)).mean();
  #return F.cross_entropy(prediction, target)
def calc_grad(xb, yb, model):
  preds = model(xb)
  loss = mnist_loss(preds, yb)
  #print("loss: ", loss)
  loss.backward()

In [66]:
# need 7840 weights. Have "10 additional neurons in the second layer"
w1, w2 = init_params((784, 16), 0.1), init_params((16, 10), 0.1)
b1, b2 = init_params(16, 0.1), init_params(10, 0.1)
#w1, w2 = w1.to(device), w2.to(device)
#b1, b2 = b1.to(device), b2.to(device)
# weights = weights.to(device)
# bias = bias.to(device)
def linear1(x):
  x = x@w1 + b1
  x = F.relu(x)
  x = x@w2 + b2
  return x
w1[:2, :2], w2[:2,:2], b1[1], b2[1]

(tensor([[ 0.1989,  0.1225],
         [ 0.2304, -0.0323]], grad_fn=<SliceBackward0>),
 tensor([[ 0.0184, -0.0527],
         [ 0.0886,  0.0508]], grad_fn=<SliceBackward0>),
 tensor(-0.2061, grad_fn=<SelectBackward0>),
 tensor(-0.0958, grad_fn=<SelectBackward0>))

In [70]:
opt = BasicOptimizer((w1, w2, b1, b2), 0.1)
#train_epoch(linear1)
xb, yb = next(iter(dl_train))
xb.shape,yb.shape
xb = xb.view(xb.size(0), -1)
calc_grad(xb, yb, linear1)
print(w1[:2, :2], '\n', w2[:2,:2], '\n', b1[1], '\n', b2[1], '\n', w1.grad[143])
opt.step()
print("AFTER=====")
print(w1[:2, :2], '\n', w2[:2,:2], '\n', b1[1], '\n', b2[1], '\n', w1.grad[143])
# for i in range(w1.size(0)):
#   if (w1.grad[i,].sum() != 0):
#     print(i, " ", w1.grad[i,])
opt.zero_grad()
print(yb)
for i in range(4):
  xb, yb = next(iter(dl_train))
  xb = xb.view(xb.size(0), -1)
  print(xb.shape)
  calc_grad(xb, yb, linear1)
  opt.step()
  opt.zero_grad()
  print(validate_epoch(linear1), end= " ")


tensor([[ 0.1989,  0.1225],
        [ 0.2304, -0.0323]], grad_fn=<SliceBackward0>) 
 tensor([[ 0.0150, -0.0468],
        [ 0.0988,  0.0474]], grad_fn=<SliceBackward0>) 
 tensor(-0.2086, grad_fn=<SelectBackward0>) 
 tensor(-0.1034, grad_fn=<SelectBackward0>) 
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
AFTER=====
tensor([[ 0.1989,  0.1225],
        [ 0.2304, -0.0323]], grad_fn=<SliceBackward0>) 
 tensor([[ 0.0135, -0.0509],
        [ 0.0938,  0.0442]], grad_fn=<SliceBackward0>) 
 tensor(-0.2126, grad_fn=<SelectBackward0>) 
 tensor(-0.1126, grad_fn=<SelectBackward0>) 
 tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([8, 0, 3, 5, 5, 9, 7, 3, 4, 5, 4, 7, 6, 3, 5, 2])
torch.Size([16, 784])
0.115 torch.Size([16, 784])
0.1171 torch.Size([16, 784])
0.1192 torch.Size([16, 784])
0.1339 

In [69]:
def train_epoch(model):
  for xb, yb in dl_train:
    xb = xb.view(xb.size(0), -1)
    yb = yb.view(-1)
    calc_grad(xb, yb, model)
    opt.step()
    opt.zero_grad()

def batch_accuracy(preds, yb): # xb is pure logits
  _,preds_class = torch.max(preds, dim=1)
  return (preds_class == yb).float().mean()

def validate_epoch(model):
  accs = [batch_accuracy(model(xb.view(xb.size(0), -1)), yb) for xb, yb in dl_valid]
  return round(torch.stack(accs).mean().item(), 4)

def train_model(model, epochs):
  for i in range(epochs):
    train_epoch(model)
    print(validate_epoch(model), end=' ')

In [83]:
opt = BasicOptimizer((w1, w2, b1, b2), 0.1)
#train_epoch(linear1)
# xb, yb = next(iter(dl_train))
# xb.shape,yb.shape
# xb = xb.view(xb.size(0), -1)
train_model(linear1, 10)

# opt.step()
# print("THis is w1: ", w1)
# opt.zero_grad()
# print("b1.grad: ", b1.grad)
# out = linear1(xb).sum()
# print("Sum: ", out)
# opt.zero_grad()
#calc_grad(xb, yb, linear1)
# opt.step()
# opt.zero_grad()
# preds = linear1(xb)
# loss = mnist_loss(preds, yb)
# loss, yb.shape, preds, softmax(preds)
# print("Loss: ", loss)
# loss.backward()
#calc_grad(xb, yb, linear1)
#print("In optimizer: this is weights ", w2.grad)
#print("loss after: ", loss)

0.9541 0.9532 0.9545 0.9516 0.9526 0.9492 0.9549 0.954 0.9546 0.9547 

In [82]:
w1[:2,:2], w2[:2,:2], b1[:2], b2[:2]

(tensor([[ 0.1989,  0.1225],
         [ 0.2304, -0.0323]], grad_fn=<SliceBackward0>),
 tensor([[-1.3991,  0.7142],
         [ 1.0047,  0.3635]], grad_fn=<SliceBackward0>),
 tensor([1.4918, 0.2306], grad_fn=<SliceBackward0>),
 tensor([ 0.1015, -0.8929], grad_fn=<SliceBackward0>))

In [85]:
w1[:2,:2], w2[:2,:2], b1[:2], b2[:2]

(tensor([[ 0.1989,  0.1225],
         [ 0.2304, -0.0323]], grad_fn=<SliceBackward0>),
 tensor([[-1.6673,  0.8927],
         [ 1.1866,  0.4271]], grad_fn=<SliceBackward0>),
 tensor([1.8834, 0.3256], grad_fn=<SliceBackward0>),
 tensor([ 0.2119, -1.3242], grad_fn=<SliceBackward0>))

In [84]:
model = nn.Sequential(
    nn.Linear(28*28, 16),
    nn.ReLU(),
    nn.Linear(16, 10)
).to(device)


loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


def accuracy(model, dataloader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            xb = xb.view(xb.size(0), -1)
            preds = model(xb)
            pred_labels = preds.argmax(dim=1)
            correct += (pred_labels == yb).sum().item()
            total += yb.size(0)
    return correct / total

epochs = 5
for epoch in range(epochs):
    model.train()
    for xb, yb in dl_train:
        xb, yb = xb.to(device), yb.to(device)
        xb = xb.view(xb.size(0), -1)  # Flatten the image

        preds = model(xb)
        loss = loss_fn(preds, yb)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    val_acc = accuracy(model, dl_valid)
    print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_acc:.4f}")

Epoch 1/5, Validation Accuracy: 0.9035
Epoch 2/5, Validation Accuracy: 0.9159


KeyboardInterrupt: 

In [67]:
import torch

# Your init function
def init_params(size, std=1.0):
    return (torch.randn(size) * std).requires_grad_()

# Initialize weights
w1 = init_params((784, 16))
b1 = init_params((16,))

# Dummy input (batch of 16 images, flattened to 784)
xb = torch.randn(16, 784)

# Forward pass
def linear1(x):
    return x @ w1 + b1

# Compute output and backward
out = linear1(xb).sum()
out.backward()

# Print grad
print("requires_grad:", w1.requires_grad)  # should be True
print("grad norm:", w1.grad.norm())        # should be > 0
print("grad sample:", w1.grad[:2, :4])     # small piece of grad

requires_grad: True
grad norm: tensor(455.5259)
grad sample: tensor([[0.7173, 0.7173, 0.7173, 0.7173],
        [2.3409, 2.3409, 2.3409, 2.3409]])
